In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os

from snorkel import SnorkelSession
session = SnorkelSession()

# Here, we just set how many documents we'll process for automatic testing- you can safely ignore this!
n_docs = 500 if 'CI' in os.environ else 2591

from snorkel.parser import TSVDocPreprocessor

doc_preprocessor = TSVDocPreprocessor('data/sample_indicator.tsv', max_docs=n_docs)

from snorkel.parser.spacy_parser import Spacy
from snorkel.parser import CorpusParser

corpus_parser = CorpusParser(parser=Spacy())
%time corpus_parser.apply(doc_preprocessor, countm=n_docs)
from snorkel.models import Document, Sentence

print("Documents:", session.query(Document).count())
print("Sentences:", session.query(Sentence).count())

Clearing existing...
Running UDF...


OSError: File or directory not found: data/sample_indicator.tsv

Documents: 0
Sentences: 0


In [2]:
from snorkel.models import candidate_subclass
from snorkel.candidates import Ngrams, CandidateExtractor
from snorkel.matchers import PersonMatcher, LexiconMatcher

Indicator = candidate_subclass('Indicator', ['trend', 'indicator'])

#ngrams         = Ngrams(n_max=7)
#person_matcher = PersonMatcher(longest_match_only=True) # should use a regex to only get nouns?
#cand_extractor = CandidateExtractor(Spouse, [ngrams, ngrams], [person_matcher, person_matcher])

indicator_ngrams = Ngrams(n_max=1)
indicator_matcher = LexiconMatcher(longest_match_only=True)
indicator_extractor = CandidateExtractor(Indicator, [indicator_ngrams, indicator_ngrams], [trend_matcher, indicator_matcher])

/home/pegah/anaconda3/envs/snorkel/lib/python3.6/site-packages/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array




    Only loading the 'en' tokenizer.



NameError: name 'trend_matcher' is not defined

In [ ]:
from snorkel.models import Document
from util import number_of_people

docs = session.query(Document).order_by(Document.name).all()

train_sents = set()
dev_sents   = set()
test_sents  = set()

for i, doc in enumerate(docs):
    for s in doc.sentences:
            if i % 4 == 2:
                dev_sents.add(s)
            elif i % 4 == 3:
                test_sents.add(s)
            else:
                train_sents.add(s)

%%time
for i, sents in enumerate([train_sents, dev_sents, test_sents]):
    print(i)
   # cand_extractor.apply(sents, split=i)
    #print("Number of candidates:", session.query(Spouse).filter(Spouse.split == i).count())
    indicator_extractor.apply(sents, split=i)
    print("Number of (trend) candidates:", session.query(Indicator).filter(Indicator.split == i).count())

In [ ]:
from my_util import load_external_trend_labels
#, load_external_trend_labels
%time missed = load_external_trend_labels(session, Indicator, annotator_name='gold')